In [39]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'anomaly_detect_for_all_participants (1).gsheet'
 anomaly_detect_for_all_participants.csv
 anomaly_detect_for_all_participants.gsheet
 anomaly_detect_for_INS-W_1.csv
 anomaly_detect_for_INS-W_2.csv
 anomaly_detect_for_INS-W_2.gsheet
 anomaly_detection_combined_features.csv
 anomaly_detection.csv
 anomaly_detection.gsheet
 anomaly_detection_isolation_forest.csv
 anomaly_detection_isolation_forest.gsheet
 anomaly_mood_for_INS-W_1.csv
 anomaly_mood_for_INS-W_2.csv
 anomaly_mood_for_INS-W_2.gsheet
 hist_labelcounts_neurips.pdf
 INS-W_1_sleep_step_screen_allday_100_participants.csv
 INS-W_2_sleep_step_screen_allday_100_participants.csv
 INS-W_2_sleep_step_screen_allday_15_participants.csv
 INS-W_2_sleep_step_screen_allday_50_participants.csv
 INS-W_3_sleep_step_screen_allday_100_participants.csv
 INS-W_3_sleep_step_screen_allday_15_participants.csv
 INS-W_4_sleep_

In [40]:
%cd '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset'

/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset


In [41]:
# prompt: write a code to read this csv /content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/sleep_step_screen_allday_15_participants.csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import shap

df_INS_W1 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_1_sleep_step_screen_allday_100_participants.csv')
df_INS_W2 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_2_sleep_step_screen_allday_100_participants.csv')
df_INS_W3 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_3_sleep_step_screen_allday_100_participants.csv')
df_INS_W4 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_4_sleep_step_screen_allday_100_participants.csv')


In [42]:
print(df_INS_W1.shape)
print(df_INS_W2.shape)
print(df_INS_W3.shape)
print(df_INS_W4.shape)

(7313, 199)
(7580, 199)
(7894, 199)
(8197, 199)


In [47]:
import pandas as pd

# Paths to the input CSV files
file_paths = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_4_sleep_step_screen_allday_100_participants.csv'
]

# dep_data_path = [
#     '/content/drive/MyDrive/CSE 570 Wireless Project/dataset/globem-dataset-multi-year-datasets-for-longitudinal-human-behavior-modeling-generalization-1.1/INS-W_1/SurveyData/dep_weekly.csv'
# ]

# Define bins and labels for classification
bins = [0, 70, 84, 90, 100]  # Bins for sleep efficiency
labels = [0, 1, 2, 3]  # Bin labels for classification

# Initialize a list to collect processed data
all_anomaly_data = []

for file_path in file_paths:
    # Load the dataset
    sleep_data = pd.read_csv(file_path)

    # Apply binning to the target column
    sleep_data['efficiency_bin'] = pd.cut(
        sleep_data['f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday'],
        bins=bins,
        labels=labels,
        right=False,
    ).astype("category")

    # Add previous bin as a feature for temporal dependency
    sleep_data['previous_bin'] = sleep_data.groupby('pid')['efficiency_bin'].shift(1)

    # Drop rows where `efficiency_bin` or `previous_bin` is NaN
    sleep_data = sleep_data.dropna(subset=['efficiency_bin', 'previous_bin'])

    # Convert bins to integers for calculation
    sleep_data['efficiency_bin'] = sleep_data['efficiency_bin'].astype(int)
    sleep_data['previous_bin'] = sleep_data['previous_bin'].astype(int)

    # Calculate bin change for each row
    sleep_data['bin_change'] = sleep_data['efficiency_bin'] - sleep_data['previous_bin']

    # Define the anomaly column (1 for decrease, 0 for same or increase)
    sleep_data['anomaly'] = sleep_data['bin_change'].apply(lambda x: 1 if x < 0 else 0)

    # Append the processed data to the list
    # all_anomaly_data.append(sleep_data[['pid', 'date', 'efficiency_bin', 'previous_bin', 'bin_change', 'anomaly']])
    all_anomaly_data.append(sleep_data)

# Combine all processed data into a single DataFrame
combined_anomaly_data = pd.concat(all_anomaly_data, ignore_index=True)

# Save the result to a CSV file
combined_anomaly_data.to_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_INS-W_4.csv', index=False)

print("Anomaly detection results for all participants have been saved to 'anomaly_detect_for_all_participants.csv'.")


Anomaly detection results for all participants have been saved to 'anomaly_detect_for_all_participants.csv'.


In [48]:
combined_anomaly_data

,pid,date,f_screen:phone_screen_rapids_avgdurationunlock:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_living:allday,...,f_steps:fitbit_steps_summary_rapids_mediansumsteps_norm:allday,f_steps:fitbit_steps_summary_rapids_minsumsteps:allday,f_steps:fitbit_steps_summary_rapids_minsumsteps_norm:allday,f_steps:fitbit_steps_summary_rapids_stdsumsteps:allday,f_steps:fitbit_steps_summary_rapids_stdsumsteps_dis:allday,f_steps:fitbit_steps_summary_rapids_stdsumsteps_norm:allday,efficiency_bin,previous_bin,bin_change,anomaly
0,INS-W_1000#INS-W_4,2021-03-30,3.737923,1.267428,0.000000,1.09760,0.000000,4.224996,0.000000,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
1,INS-W_1000#INS-W_4,2021-03-31,1.829372,1.267428,0.000000,1.09760,0.000000,0.324006,-0.508922,0.324006,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
2,INS-W_1000#INS-W_4,2021-04-01,1.741264,1.267428,0.000000,1.09760,0.000000,4.224996,0.000000,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
3,INS-W_1000#INS-W_4,2021-04-02,2.590941,1.267428,0.000000,1.09760,0.000000,4.224996,0.000000,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
4,INS-W_1000#INS-W_4,2021-04-03,3.196991,1.267428,0.000000,1.09760,0.000000,4.224996,0.000000,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7790,INS-W_996#INS-W_4,2021-06-17,6.670411,1.267428,0.000000,1.09760,0.000000,6.884741,0.497296,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
7791,INS-W_996#INS-W_4,2021-06-18,5.367919,1.267428,0.000000,1.09760,0.000000,5.750200,0.285775,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
7792,INS-W_996#INS-W_4,2021-06-19,3.297271,1.999967,0.210588,3.22384,0.510949,5.265091,0.195333,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0
7793,INS-W_996#INS-W_4,2021-06-20,6.377686,3.121155,0.478039,1.09760,0.000000,5.827912,0.300264,3.170859,...,0.0,4851.5,0.0,NaN,NaN,NaN,3,3,0,0


In [49]:
file_paths = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_INS-W_4.csv'
]

dep_data_path = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/dataset/globem-dataset-multi-year-datasets-for-longitudinal-human-behavior-modeling-generalization-1.1/INS-W_4/SurveyData/dep_weekly.csv'
]

dep_df = pd.read_csv(dep_data_path[0])
dep_df

,Unnamed: 0,pid,date,phq4,dep,BDI2,dep_weekly_subscale,anx_weekly_subscale,dep_weeklysubscale_endterm_merged
0,0,INS-W_1000,2021-04-04,2.0,False,NaN,False,False,False
1,1,INS-W_1000,2021-04-11,3.0,True,NaN,False,False,False
2,2,INS-W_1000,2021-04-18,6.0,True,NaN,True,False,True
3,3,INS-W_1000,2021-04-25,1.0,False,NaN,False,False,False
4,4,INS-W_1000,2021-05-02,5.0,True,NaN,True,False,True
...,...,...,...,...,...,...,...,...,...
2169,2169,INS-W_999,2021-05-30,0.0,False,NaN,False,False,False
2170,2170,INS-W_999,2021-05-31,0.0,False,NaN,False,False,False
2171,2171,INS-W_999,2021-06-06,0.0,False,NaN,False,False,False
2172,2172,INS-W_999,2021-06-13,0.0,False,NaN,False,False,False


In [50]:
import pandas as pd
from sklearn.impute import KNNImputer

# Define file paths
file_paths = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_INS-W_4.csv'
]

dep_data_path = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/dataset/globem-dataset-multi-year-datasets-for-longitudinal-human-behavior-modeling-generalization-1.1/INS-W_4/SurveyData/dep_weekly.csv'
]

# Load datasets
main_df = pd.read_csv(file_paths[0])
dep_df = pd.read_csv(dep_data_path[0])

# Drop the "Unnamed: 0" column from dep_df
dep_df = dep_df.loc[:, dep_df.columns != 'Unnamed: 0']

# Process the `pid` column in main_df to match the format in dep_df
main_df['pid'] = main_df['pid'].str.split('#').str[0]

# Convert `date` columns to datetime format
main_df['date'] = pd.to_datetime(main_df['date'])
dep_df['date'] = pd.to_datetime(dep_df['date'])

# Perform an outer merge on `pid` and `date` to ensure no rows are dropped
merged_df = pd.merge(main_df, dep_df, on=['pid', 'date'], how='outer')

# # Columns to impute
# columns_to_impute = ['dep'] #, 'feel_depressed']

# # Ensure the columns are present in the dataframe
# merged_df[columns_to_impute] = merged_df[columns_to_impute].astype(float)

# imputer = KNNImputer(n_neighbors=3)
# imputed_data = imputer.fit_transform(merged_df[columns_to_impute])

# # Replace the original columns with the imputed data as integers
# merged_df[columns_to_impute] = pd.DataFrame(imputed_data, columns=columns_to_impute).round(0).astype(bool)

# # Display the merged dataframe
merged_df.to_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_mood_for_INS-W_4.csv', index=False)

merged_df

# print("Anomaly detection results for all participants have been saved to 'anomaly_mood_for_INS-W_2.csv'.")



,pid,date,f_screen:phone_screen_rapids_avgdurationunlock:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_living:allday,...,efficiency_bin,previous_bin,bin_change,anomaly,phq4,dep,BDI2,dep_weekly_subscale,anx_weekly_subscale,dep_weeklysubscale_endterm_merged
0,INS-W_1000,2021-03-30,3.737923,1.267428,0.0,1.0976,0.0,4.224996,0.000000,3.170859,...,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,INS-W_1000,2021-03-31,1.829372,1.267428,0.0,1.0976,0.0,0.324006,-0.508922,0.324006,...,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,INS-W_1000,2021-04-01,1.741264,1.267428,0.0,1.0976,0.0,4.224996,0.000000,3.170859,...,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,INS-W_1000,2021-04-02,2.590941,1.267428,0.0,1.0976,0.0,4.224996,0.000000,3.170859,...,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,INS-W_1000,2021-04-03,3.196991,1.267428,0.0,1.0976,0.0,4.224996,0.000000,3.170859,...,3.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8929,INS-W_999,2021-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
8930,INS-W_999,2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
8931,INS-W_999,2021-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
8932,INS-W_999,2021-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False


In [34]:
import pandas as pd

# Define file paths
file_paths = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_INS-W_2.csv'
]

dep_data_path = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/dataset/globem-dataset-multi-year-datasets-for-longitudinal-human-behavior-modeling-generalization-1.1/INS-W_2/SurveyData/dep_weekly.csv'
]

# Load datasets
main_df = pd.read_csv(file_paths[0])
dep_df = pd.read_csv(dep_data_path[0])

# Drop the "Unnamed: 0" column from dep_df
dep_df = dep_df.loc[:, dep_df.columns != 'Unnamed: 0']

# Process the `pid` column in main_df to match the format in dep_df
main_df['pid'] = main_df['pid'].str.split('#').str[0]

# Convert `date` columns to datetime format
main_df['date'] = pd.to_datetime(main_df['date'])
dep_df['date'] = pd.to_datetime(dep_df['date'])

# Perform an outer merge on `pid` and `date` without suffix issues
merged_df = pd.merge(
    main_df,
    dep_df,
    on=['pid', 'date'],
    how='outer'
)

# Ensure `date` from `main_df` is preserved
merged_df['date'] = merged_df['date']

# Display the resulting dataframe
merged_df


,pid,date,f_screen:phone_screen_rapids_avgdurationunlock:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_living:allday,...,efficiency_bin,previous_bin,bin_change,anomaly,phq4,dep,BDI2,dep_weekly_subscale,anx_weekly_subscale,dep_weeklysubscale_endterm_merged
0,INS-W_300,2019-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,False,NaN,False,False,False
1,INS-W_300,2019-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,True,NaN,False,False,False
2,INS-W_300,2019-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,True,NaN,False,False,False
3,INS-W_300,2019-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,True,NaN,True,False,True
4,INS-W_300,2019-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,True,NaN,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8626,INS-W_572,2019-05-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,True,NaN,False,True,False
8627,INS-W_572,2019-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,False,NaN,False,False,False
8628,INS-W_572,2019-05-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,True,NaN,True,True,True
8629,INS-W_572,2019-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.0,True,NaN,False,True,False


In [52]:
import pandas as pd
import os

# List of file paths or directory containing CSV files
file_paths = [
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_mood_for_INS-W_2.csv',
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_mood_for_INS-W_3.csv',
    '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_mood_for_INS-W_4.csv',
    # Add more file paths as needed
]

# Initialize an empty list to store individual dataframes
dataframes = []

# Loop through file paths and read each CSV into a DataFrame
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all DataFrames vertically
merged_df = pd.concat(dataframes, axis=0, ignore_index=True)

# Save the merged DataFrame to a new CSV file (optional)
merged_df.to_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_mood_for_all_participants.csv', index=False)

# Display the merged DataFrame
merged_df


,pid,date,f_screen:phone_screen_rapids_avgdurationunlock:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_living:allday,...,efficiency_bin,previous_bin,bin_change,anomaly,phq4,dep,BDI2,dep_weekly_subscale,anx_weekly_subscale,dep_weeklysubscale_endterm_merged
0,INS-W_300,2019-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,False,NaN,False,False,False
1,INS-W_300,2019-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,True,NaN,False,False,False
2,INS-W_300,2019-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,True,NaN,False,False,False
3,INS-W_300,2019-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,True,NaN,True,False,True
4,INS-W_300,2019-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,True,NaN,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25653,INS-W_999,2021-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
25654,INS-W_999,2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
25655,INS-W_999,2021-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
25656,INS-W_999,2021-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,False,NaN,False,False,False
